# Simulations of Survival rate

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import openrouteservice

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Own Classes
from Simulation_Routing import *
from Simulating import *
from Simulation_Routing_Matrix import *
from Simulation_Routing_Matrix_copy import *
from Simulation_Routing_Matrix_Batch import *
from tqdm import tqdm

Initialize the class using the ip of the server.

In [3]:
ip = "35.159.16.25"

### New function version
Wrapped a function around so that we can better automate the simulations. Can also better keep track of what parameters we have specified and which have already been tried.

In [7]:
## NEW function version (old pre-function version below)

def simulation_run(ip=ip,              ## server IP, must request from Ruben (or others)
                   num_sim = 3,        ## number of simulation runs to perform for the given parameter values
                   max_num_resp = 100, ## maximum number of responders that can be considered
                   open_hr = 14.0,     ## given opening hour (choose one of 2.0, 8.0, 14.0, 20.0)
                   prop_cfr = 0.1,    ## proportion of the total population who are CFRs, determines the number sampled
                   aed_filter = ["Yes"]): ## which AEDs to consider (choose None to allow all)
    Simulations = Simulation(ip)
    RoutingMatrixCopy = RoutingSimulationMatrixBatch(ip)

    ## OTHER PARAMETERS determined by our function input
    
    # Likelihood of a responder declining the call to action
    # varies according to whether it is the day, evening, or night
    if 7.0 <= open_hr < 17.0:
        decline_rate = 0.7
    elif 17.0 <= open_hr < 23.0:
        decline_rate = 0.65
    else:
        decline_rate = 0.75
    
    # time of the day
    time_of_day = "day" if 8.0 <= open_hr <= 18.0 else "night"
    
    df_final = pd.DataFrame()

    print("Input values: \n opening_hour: " + str(open_hr) + 
      "\n decline_rate: " + str(decline_rate) + 
      "\n max_number_responder: " + str(max_num_resp) + 
      "\n filter_values: " + str(aed_filter) +
      "\n time_of_day: " + str(time_of_day) +
      "\n proportion: " + str(prop_cfr))

    x = 1
    df_final = []
    while x <= num_sim:
        try:
            df = Simulations.simulation_run(
                decline_rate, max_num_resp, open_hr,
                aed_filter, time_of_day, prop_cfr
            )
            df['filter_values'] = ','.join(aed_filter) if isinstance(aed_filter, list) else aed_filter
            df['opening_hour'] = open_hr
            df['decline_rate'] = decline_rate
            df['max_number_responder'] = max_num_resp
            df['proportion_of_CFR'] = prop_cfr
            # Rename all columns except 'patient_loc'
            suffix = f"_run{x}"
            df_renamed = df.rename(
                columns={col: f"{col}{suffix}" for col in df.columns if col != 'patient_loc'}
            )
    
            # Merge into the master DataFrame
            df_final.append(df_renamed)
    
        except Exception as e:
            print(f"[Warning] Simulation {x} failed: {e}. Retrying...")
        
        x += 1

    ## combine the run results with previous runs' results
    df_final = pd.concat(df_final, ignore_index=True)
    
    ## Build a string-safe version of parameters
    aed_str = "all" if aed_filter is None else "-".join(aed_filter)
    filename = (
        f"sim_results_"
        f"openhr{open_hr}_"
        f"cfr{prop_cfr}_"
        f"maxresp{max_num_resp}_"
        f"numsim{num_sim}_"
        f"aed{aed_str}.csv"
    )
    
    # Save the results
    df_final.to_csv(filename, index=False)
    print(f"Results saved to {filename}")

In [8]:
opening_hours = [2.0, 8.0, 14.0, 20.0]
for i in opening_hours:
    simulation_run(open_hr = i, prop_cfr = 0.02)

Input values: 
 opening_hour: 2.0
 decline_rate: 0.75
 max_number_responder: 100
 filter_values: ['Yes']
 time_of_day: night
 proportion: 0.02


100%|██████████| 388/388 [06:04<00:00,  1.06it/s]


Results saved to sim_results_openhr2.0_cfr0.02_maxresp100_numsim3_aedYes.csv
Input values: 
 opening_hour: 8.0
 decline_rate: 0.7
 max_number_responder: 100
 filter_values: ['Yes']
 time_of_day: day
 proportion: 0.02


100%|██████████| 388/388 [07:53<00:00,  1.22s/it]


Results saved to sim_results_openhr8.0_cfr0.02_maxresp100_numsim3_aedYes.csv
Input values: 
 opening_hour: 14.0
 decline_rate: 0.7
 max_number_responder: 100
 filter_values: ['Yes']
 time_of_day: day
 proportion: 0.02


100%|██████████| 388/388 [10:06<00:00,  1.56s/it]


Results saved to sim_results_openhr14.0_cfr0.02_maxresp100_numsim3_aedYes.csv
Input values: 
 opening_hour: 20.0
 decline_rate: 0.65
 max_number_responder: 100
 filter_values: ['Yes']
 time_of_day: night
 proportion: 0.02


100%|██████████| 388/388 [07:02<00:00,  1.09s/it]

Results saved to sim_results_openhr20.0_cfr0.02_maxresp100_numsim3_aedYes.csv


In [41]:

# ## OLD pre-function

# ip = "18.193.109.153"
# Simulations = Simulation(ip)
# RoutingMatrixCopy = RoutingSimulationMatrixBatch(ip)

# # Number of times the simulations are run to later get an average survivability rate
# NO_SIMULATIONS = 6

# ## PARAMETERS WE SET
# # Maximal number of responder contacted
# max_number_responder = 100
# # Time of the incident
# ## choose one of the following: 2.0, 8.0, 14.0, 20.0
# opening_hour = 14.0
# # proportion of CFR of the total (ex.: 0.01 = 1%)
# proportion = 0.01
# # list of values that are in the available column
# filter_values = ["Yes"]

# ## OTHER PARAMETERS
# # Likelihood of a responder declining the call to action
# # varies according to whether it is the day, evening, or night
# if 7.0 <= opening_hour < 17.0:
#     decline_rate = 0.7
# elif 17.0 <= opening_hour < 23.0:
#     decline_rate = 0.65
# else:
#     decline_rate = 0.75

# # time of the day
# time_of_day = "day" if 8.0 <= opening_hour <= 18.0 else "night"

# df_final = pd.DataFrame()

# print("Input values: \n opening_hour: " + str(opening_hour) + 
#       "\n decline_rate: " + str(decline_rate) + 
#       "\n max_number_responder: " + str(max_number_responder) + 
#       "\n filter_values: " + str(filter_values) +
#       "\n time_of_day: " + str(time_of_day) +
#       "\n proportion: " + str(proportion))

# df_final = pd.DataFrame()

# x = 0
# df_final = []
# while x < NO_SIMULATIONS:
#     try:
#         df = Simulations.simulation_run(
#             decline_rate, max_number_responder, opening_hour,
#             filter_values, time_of_day, proportion
#         )
#         df['filter_values'] = ','.join(filter_values) if isinstance(filter_values, list) else filter_values
#         df['opening_hour'] = opening_hour
#         df['decline_rate'] = decline_rate
#         df['max_number_responder'] = max_number_responder
#         df['proportion_of_CFR'] = proportion
#         # Rename all columns except 'patient_loc'
#         suffix = f"_run{x+1}"
#         df_renamed = df.rename(
#             columns={col: f"{col}{suffix}" for col in df.columns if col != 'patient_loc'}
#         )

#         # Merge into the master DataFrame
#         df_final.append(df_renamed)

#     except Exception as e:
#         print(f"[Warning] Simulation {x+1} failed: {e}. Retrying...")

# df_final = pd.concat(df_final, ignore_index=True)


Input values: 
 opening_hour: 14.0
 decline_rate: 0.7
 max_number_responder: 100
 filter_values: ['Yes']
 time_of_day: day
 proportion: 0.01


 24%|██▍       | 94/388 [01:10<03:40,  1.33it/s]


KeyboardInterrupt: 

In [30]:
# SAVE THE DATAFRAME!!!
df.to_csv("Data/sim_runs/YOUR_FILENAME_HERE.csv")

## Debugging


In [29]:
## PARAMETERS WE SET
# Maximal number of responder contacted
max_number_responder = 1000
# Time of the incident
## choose one of the following: 2.0, 8.0, 14.0, 20.0
opening_hour = 14.0
# proportion of CFR of the total (ex.: 0.01 = 1%)
proportion = 0.02


print("Input values: \n opening_hour: " + str(opening_hour) + 
      "\n decline_rate: " + str(decline_rate) + 
      "\n max_number_responder: " + str(max_number_responder) + 
      "\n filter_values: " + str(filter_values) +
      "\n time_of_day: " + str(time_of_day) +
      "\n proportion: " + str(proportion))

df = Simulations.simulation_run(
            decline_rate, max_number_responder, opening_hour,
            filter_values, time_of_day, proportion
        )


Input values: 
 opening_hour: 14.0
 decline_rate: 0.7
 max_number_responder: 1000
 filter_values: ['Yes']
 time_of_day: day
 proportion: 0.02


100%|██████████| 388/388 [10:08<00:00,  1.57s/it]


In [30]:
# SAVE THE DATAFRAME!!!
df.to_csv("Data/sim_runs/debug_run_3.csv")

In [32]:
df_1 = pd.read_csv("Data/sim_runs/debug_run.csv")
df_2 = pd.read_csv("Data/sim_runs/debug_run_2.csv")

In [39]:
df["responder_loc"].value_counts()

responder_loc
No responder                               112
(4.726162579635614, 50.87427682579571)       4
(4.728076759091802, 50.8862301174485)        3
(4.6901339607145145, 50.89294406168314)      2
(4.729748648752874, 50.900059324561035)      2
                                          ... 
(4.662973834193056, 50.86121395811966)       1
(4.702060059438565, 50.88841222083602)       1
(4.724808214661289, 50.92769673861054)       1
(4.708022109575111, 50.8769896670177)        1
(4.730412749638898, 50.93259863929417)       1
Name: count, Length: 239, dtype: int64